In [1]:
!pip install -q transformers sentence-transformers torch pypdf accelerate

import torch
from sentence_transformers import SentenceTransformer, util
from transformers import pipeline
from pypdf import PdfReader
import logging
import os

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.9/328.9 kB 11.6 MB/s eta 0:00:00


In [6]:
# Silencia logs desnecessários
logging.getLogger("transformers").setLevel(logging.ERROR)

def carregar_pdf_zao(caminho_arquivo):
    print(f"Lendo PDF: {caminho_arquivo}...")
    try:
        reader = PdfReader(caminho_arquivo)
    except:
        return []

    texto_total = ""
    for page in reader.pages:
        texto_total += page.extract_text()

    texto_limpo = texto_total.replace('\n', ' ').replace('  ', ' ')

    tamanho_pedaco = 1200
    overlap = 200

    pedacos = []
    for i in range(0, len(texto_limpo), tamanho_pedaco - overlap):
        pedaco = texto_limpo[i:i+tamanho_pedaco]
        if len(pedaco) > 50:
            pedacos.append(pedaco)

    print(f"Base pronta! {len(pedacos)} trechos longos gerados.")
    return pedacos

from google.colab import drive
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

meu_caminho = '/content/drive/MyDrive/6 semestre/IA/Atividade 11/baseConhecimentoAtividade11.pdf'
base_de_conhecimento = carregar_pdf_zao(meu_caminho)

print("Baixando TinyLlama... (Pode demorar uns 2 minutinhos)")

modelo_busca = SentenceTransformer('all-MiniLM-L6-v2')
embeddings_base = modelo_busca.encode(base_de_conhecimento, convert_to_tensor=True)

pipe = pipeline("text-generation",
                model="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
                torch_dtype=torch.bfloat16,
                device_map="auto")

def sistema_rag_pro(pergunta):
    embedding_pergunta = modelo_busca.encode(pergunta, convert_to_tensor=True)
    resultados = util.semantic_search(embedding_pergunta, embeddings_base, top_k=2)

    contexto = ""
    for hit in resultados[0]:
        idx = hit['corpus_id']
        contexto += base_de_conhecimento[idx] + "\n---\n"

    print(f"\n--- Pergunta: {pergunta} ---")
    messages = [
        {
            "role": "system",
            "content": """Você é um especialista em Compliance e Ética da Avengers.
            DIRETRIZES OBRIGATÓRIAS:
            1. IDIOMA: Responda APENAS usando Português do Brasil.
            2. ATENÇÃO ÀS PROIBIÇÕES: Verifique se o texto menciona "Proibido", "Vedado", "Não aceitamos" ou "Situações Proibidas" antes da informação.
            3. Não forneça respostas ambiguas, quando não tiver o contexto necessario responda que se deve falar com o squad de compliance.
            4. tudo o que for "estritamente profissional" nao deve nunca ser usado para algo que não seja o trabalho ou envolva o ambito profissional.
            5. Seja direto e baseie-se APENAS no contexto abaixo."""
        },
        {
            "role": "user",
            "content": f"Contexto:\n{contexto}\n\nPergunta: {pergunta}"
        },
    ]

    prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

    outputs = pipe(prompt, max_new_tokens=300, do_sample=True, temperature=0.1, top_k=50, top_p=0.95)

    resposta_limpa = outputs[0]["generated_text"].split("<|assistant|>")[-1].strip()
    return resposta_limpa

Lendo PDF: /content/drive/MyDrive/6 semestre/IA/Atividade 11/baseConhecimentoAtividade11.pdf...
Base pronta! 9 trechos longos gerados.
Baixando TinyLlama... (Pode demorar uns 2 minutinhos)


In [ ]:
print("R1:", sistema_rag_pro("Posso aceitar brindes de parceiros?"), '\n\n')
print("R2:", sistema_rag_pro("Posso postar numa rede social novidades sobre um projeto que esta em desenvolvimento?"), '\n\n')
print("R3:", sistema_rag_pro("Posso usar o notebook do trabalho para uso pessoal?"), '\n\n')


--- Pergunta: Posso aceitar brindes de parceiros? ---
R1: Sure, you can accept gifts or benefits from partners as long as they are strictly professional and do not involve your work or the professional sphere. This includes things like parcel deliveries, discounts, and hospitality. However, you should base your decision on the context and ensure that it is aligned with the Avengers' values and principles. 



--- Pergunta: Posso postar numa rede social novidades sobre um projeto que esta em desenvolvimento? ---


In [ ]:
print("Larissa Pereira Ibiapino \nCiencia da computacao 6an")

Larissa Pereira Ibiapino 
Ciencia da computacao 6an
